In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


In [62]:
from adamp import AdamP

class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        p_encoder,
        q_encoder
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset
        self.bert = SentenceTransformer(sbert_model_name)
        self.tokenizer = sbert_model.tokenizer

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer

        # q_seqs = tokenizer(self.dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
        q_seqs = tokenizer(self.dataset['question'], padding="max_length", truncation=True, return_tensors='pt', max_length=512)
        # p_seqs = tokenizer(self.dataset['context'], padding="max_length", truncation=True, return_tensors='pt')
        p_seqs = tokenizer(self.dataset['context'], padding="max_length", truncation=True, return_tensors='pt', max_length=512)

        train_dataset = TensorDataset(
            p_seqs['input_ids'], 
            p_seqs['attention_mask'], 
            p_seqs['token_type_ids'], 
            q_seqs['input_ids'], 
            q_seqs['attention_mask'], 
            q_seqs['token_type_ids']
            )
        
        train_dataloader = DataLoader(
            train_dataset, 
            batch_size=args.per_device_train_batch_size
            )

        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamP(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        # t_total = len(train_dataloader) * args.num_train_epochs
        # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

        for epoch, _ in enumerate(train_iterator):
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            # loss_value=0 # Accumulation할 때 진행
            losses = 0
            for step, batch in enumerate(epoch_iterator):
                self.q_encoder.train()
                self.p_encoder.train()
                
                if torch.cuda.is_available():
                    batch = tuple(t.cuda() for t in batch)

                p_inputs = {'input_ids': batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids': batch[2]
                            }
                
                q_inputs = {'input_ids': batch[3],
                            'attention_mask': batch[4],
                            'token_type_ids': batch[5]}
                
                
                p_outputs = self.p_encoder(
                    input_ids = p_inputs["input_ids"],
                    attention_mask=p_inputs["attention_mask"],
                    token_type_ids=p_inputs["token_type_ids"]
                )  # (batch_size, emb_dim)
                
                q_outputs = self.q_encoder(
                    input_ids = q_inputs["input_ids"],
                    attention_mask=q_inputs["attention_mask"],
                    token_type_ids=q_inputs["token_type_ids"]
                )  # (batch_size, emb_dim)

                # Calculate similarity score & loss
                sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

                # target: position of positive samples = diagonal element 
                targets = torch.arange(0, args.per_device_train_batch_size).long()
                if torch.cuda.is_available():
                    targets = targets.to('cuda')

                # almost same as cross entropy loss
                sim_scores = F.log_softmax(sim_scores, dim=1)
                loss = F.nll_loss(sim_scores, targets)

                losses += loss.item()
                if step % 100 == 0 :
                    print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리

                loss.backward()
                #################ACCUMULATION###############################
                # loss_value += loss
                # if (step+1) % args.gradient_accumulation_steps == 0 :
                #     optimizer.step()
                #     scheduler.step()
                #     self.q_encoder.zero_grad()
                #     self.p_encoder.zero_grad()
                #     global_step += 1
                #     print(loss_value/args.gradient_accumulation_steps)
                #     loss_value = 0
                ############################################################
                optimizer.step()
                # scheduler.step()
                self.q_encoder.zero_grad()
                self.p_encoder.zero_grad()
                global_step += 1
                
                torch.cuda.empty_cache()
                del p_inputs, q_inputs

In [63]:
from transformers import BertModel

from sentence_transformers import SentenceTransformer

class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        # self.bert = BertModel(config)
        sbert_model_name = 'KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS'
        # self.sbert_model = SentenceTransformer(sbert_model_name)
        config = SentenceTransformer(sbert_model_name)._first_module().auto_model.config # for bert token embeddings
        self.sbert_model = BertModel(config)
        self.tokenizer = sbert_model.tokenizer
        self.init_weights()
    
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        model_output = self.sbert_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        sentence_embeddings = self.mean_pooling(model_output, attention_mask)

        return sentence_embeddings

In [64]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [65]:
args = TrainingArguments(
    output_dir="dense_retrieval",
    evaluation_strategy="epoch",
    learning_rate=5e-5, # recommended learning rate is 1e-5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    weight_decay=0.01
)

In [66]:
# model_checkpoint = "klue/bert-base"
sbert_model_name = 'KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS'
sbert_model = SentenceTransformer(sbert_model_name)
config = sbert_model._first_module().auto_model.config # for bert token embeddings
config

BertConfig {
  "_name_or_path": "KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS/0_Transformer",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 40000
}

In [67]:
# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
p_encoder = BertEncoder(config).to(args.device)
q_encoder = BertEncoder(config).to(args.device)

In [68]:
# https://github.com/UKPLab/sentence-transformers/issues/794
tokenizer = sbert_model.tokenizer
tokenizer

PreTrainedTokenizerFast(name_or_path='KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS/0_Transformer', vocab_size=40000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [54]:
sbert_model = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')
encoded_input = tokenizer("내게로 와줘, 내 일상 속으로")
encoded_input

{'input_ids': [2, 12888, 5178, 3599, 5672, 16, 2340, 10815, 13726, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [69]:
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder
)
retriever.train()

0epoch loss: 3.412809371948242


In [45]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [48]:
p_encoder = retriever.p_encoder
q_encoder = retriever.q_encoder
with torch.no_grad() :
    p_encoder.eval()

    p_embs = []
    for p in tqdm(corpus) :
        p = tokenizer([p], padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
p_embs = torch.Tensor(p_embs).squeeze()

In [49]:
corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [50]:
p_embs

tensor([[ 0.8959, -0.6740,  0.8944,  ..., -0.6470,  0.8301,  0.9416],
        [ 0.8310, -0.7759,  0.8776,  ..., -0.5936,  0.8365,  0.9866],
        [ 0.8491, -0.6105,  0.8313,  ..., -0.8094,  0.8443,  0.9948],
        ...,
        [ 0.6796, -0.6418,  0.8452,  ..., -0.4144,  0.7531,  0.9738],
        [ 0.5337, -0.7360,  0.8792,  ..., -0.4087,  0.9095,  0.9789],
        [ 0.3616, -0.6952,  0.8875,  ..., -0.5358,  0.9320,  0.9518]])

In [51]:
def get_relavant_doc(query, q_encoder, p_embs, k=1) :

    with torch.no_grad() :
        q_encoder.eval()
        
        q_seqs_val = tokenizer(
                    [query],
                    padding="max_length",
                    truncation=True,
                    return_tensors="pt"
        ).to(args.device)
        q_emb = q_encoder(**q_seqs_val).to("cpu")  # (num_query=1, emb_dim)

    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

    return dot_prod_scores, rank[:k]

In [54]:
dataset['validation']['question'][0]

'처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?'

In [60]:
doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'][1], q_encoder, p_embs, k = 2)
doc_indices

tensor([47955, 22916])

In [61]:
def get_relavant_doc(queries: List, q_encoder, p_embs, k=1) :

    with torch.no_grad() :
        q_encoder.eval()
        q_embs = []
        for q in queries :
            q = tokenizer([q], padding='max_length', truncation=True, return_tensors='pt').to('cuda')
            q_emb = q_encoder(**q).to('cpu').numpy()
            q_embs.append(q_emb)
    q_embs = torch.Tensor(q_embs).squeeze()

    result = torch.matmul(q_embs, torch.transpose(p_embs, 0, 1))
    if not isinstance(result, np.ndarray) :
        result = result.cpu().detach().numpy()

    doc_scores = []
    doc_indices = []
    for i in range(result.shape[0]) :
        sorted_result = np.argsort(result[i, :][::-1])
        doc_scores.append(result[i, :][sorted_result].tolist()[:k])
        doc_indices.append(sorted_result.tolist()[:k])

    return result, doc_scores, doc_indices

In [62]:
total = []
result, doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'], q_encoder, p_embs, k = 2)


In [63]:

for idx, example in enumerate(
    tqdm(dataset['validation'], desc="Sparse retrieval: ")
):
    tmp = {
        # Query와 해당 id를 반환합니다.
        "question": example["question"],
        "id": example["id"],
        # Retrieve한 Passage의 id, context를 반환합니다.
        "context_id": doc_indices[idx],
        "context": " ".join(  # 기존에는 ' '.join()
            [corpus[pid] for pid in doc_indices[idx]]
        ),
    }
    if "context" in example.keys() and "answers" in example.keys():
        # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
        tmp["original_context"] = example["context"]
        tmp["answers"] = example["answers"]
    total.append(tmp)
cqas = pd.DataFrame(total)

In [64]:
from datasets import (
    Sequence,
    Value,
    Features,
    Dataset,
    DatasetDict,
)
f = Features(
    {
        "answers": Sequence(
            feature={
                "text": Value(dtype="string", id=None),
                "answer_start": Value(dtype="int32", id=None),
            },
            length=-1,
            id=None,
        ),
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
    }
)
datasets = DatasetDict({"validation": Dataset.from_pandas(cqas, features=f)})

## Reader

In [74]:
datasets['validation']['answers'][0]

{'answer_start': [284], 'text': ['한보철강']}

In [76]:
cqas

,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"[54872, 36706]",형의 회사엔 새로운 여직원이 들어왔고 형은 호감을 느끼게 된다. 그렇게 집에서 형은...,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"[20806, 47909]",1읍 1구 21리로 구성된다.\n\n* 문덕읍 (文德邑)\n* 상북동리 (上北洞里)...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"[2952, 8211]",제2차 세계 대전 중 붉은 군대의 전략적 작전\n동부 전선에서의 독일 작전은 특정 ...,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"[53178, 47331]","존 라잇풋(John Lightfoot, 1602년 3월 29일 - 1675년 12월...",프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"[18113, 21840]",천보(天保)는 중국 북제(北齊) 문선제(文宣帝)의 연호이다. 550년 5월에서 55...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"[23539, 39234]","김동준 (金東俊, 1981년 4월 19일 ~)은 대한민국의 전직 스타크래프트 프로게...","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"[20806, 4053]",1읍 1구 21리로 구성된다.\n\n* 문덕읍 (文德邑)\n* 상북동리 (上北洞里)...,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"[6872, 53505]",봉황대기 전국고교야구대회(鳳凰大旗全國高敎野球大會)는 한국일보사가 주최하는 전국 규모...,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"[10706, 10797]","울루그 베그는 알데라민을 알피르크(세페우스자리 베타), 알키드르(세페우스자리 에타)...",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"
